In [2]:
from ultralytics import YOLO
import cv2
import os

In [3]:
image_path = "images1/images/"
image_processed_path = "images1/images_processed/"
os.makedirs(image_processed_path, exist_ok=True)


model = YOLO("yolov10n.pt")

confidence_threshold = 0.8
people_true_number = [5, 3, 4, 4, 6, 5, 4, 4, 4, 5]
output_size = (640, 640)
results_array = []



In [4]:
for image_number in range(0, 10):
    image_file = f"{image_path}{image_number}.jpg"
    image = cv2.imread(image_file)
    
    if image is None:
        print(f"Ошибка: не удалось загрузить {image_file}")
        continue
    
    results = model(image_file)[0]

    people_detected = [box for box in results.boxes if int(box.cls) == 0 and box.conf >= confidence_threshold]

    TP = len(people_detected)
    FN = max(0, people_true_number[image_number] - TP)
    FP = 0  
    TN = 0

    results_array.append([image_number, people_true_number[image_number], TP, FP, FN])

    for box in people_detected:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        confidence = box.conf.item()

        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

        label = f"Person: {confidence:.2f}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    image_resized = cv2.resize(image, output_size)

    cv2.imshow(f"Detected People - Image {image_number}", image_resized)
    cv2.waitKey(0)

    processed_image_path = f"{image_processed_path}{image_number}_YOLO_80.jpg"
    cv2.imwrite(processed_image_path, image_resized)

cv2.destroyAllWindows()

print("Матрицы ошибок (TP, FP, FN, TN=0):")
for row in results_array:
    print(row)


image 1/1 /home/alexander/PycharmProjects/ImageProcessing/FirstTask/images1/images/0.jpg: 384x640 5 persons, 70.8ms
Speed: 2.9ms preprocess, 70.8ms inference, 30.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/alexander/PycharmProjects/ImageProcessing/FirstTask/images1/images/1.jpg: 448x640 3 persons, 40.2ms
Speed: 2.5ms preprocess, 40.2ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/alexander/PycharmProjects/ImageProcessing/FirstTask/images1/images/2.jpg: 416x640 2 persons, 2 cars, 1 potted plant, 40.8ms
Speed: 2.4ms preprocess, 40.8ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/alexander/PycharmProjects/ImageProcessing/FirstTask/images1/images/3.jpg: 448x640 4 persons, 1 car, 24.7ms
Speed: 2.7ms preprocess, 24.7ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/alexander/PycharmProjects/ImageProcessing/FirstTask/images1/images/4.jpg: 448x640 6 persons, 1 

In [5]:
total_TP = sum(row[2] for row in results_array)
total_FP = sum(row[3] for row in results_array)
total_FN = sum(row[4] for row in results_array)

print(total_TP)
print(total_FP)
print(total_FN)

31
0
13


In [6]:
precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) > 0 else 0
recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1_score:.3f}")

Precision: 1.000
Recall: 0.705
F1-Score: 0.827
